# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [3]:
library(tidyverse)

sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      # WRITE YOUR CODE HERE

      # WRITE YOUR CODE HERE
      #Try 

     # Create an empty dataframe with specified column names
      # Create the dataframe with random numbers generated from a uniform distribution
      sim_df <- data.frame(
      a = runif(sample_size, min = age_lo, max = age_hi)
      )
      sim_df <- sim_df %>%
            # simulate experience
            mutate(x = beta_xa * a + beta_x0 + rnorm(sample_size, sd = sd_x)) %>%
            #simulate comprehension
            mutate(c = beta_ca * a + beta_cx * x + beta_c0 + rnorm(sample_size, sd = sd_c))

      return (sim_df)
     
      #return(data.frame(age=age,x=x,c=c)) # it's actually bad form to have a variable named "c" in R, my bad...
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,a,x,c
,<dbl>,<dbl>,<dbl>
1,136.58260,-43.09950,55.53756
2,165.94846,78.59710,495.15331
3,154.72023,41.10360,317.17124
4,126.22973,93.01661,366.81499
5,80.12215,-11.79036,-29.18129
6,129.60322,66.08486,437.69728


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [11]:
# WRITE YOUR CODE HERE
library(mediation)

run_analysis <- function(dat_in) {
    # age is IV
    # x is the mediator
    # c is DV

    #Step 1: IV on M, regress age against reading experience
    fitM <- lm(x ~ a, dat = dat_in) 
    #Step 2: IV and M on DV, regress age and experience against comprehension
    fitY <- lm(c ~ a + x, data = dat_in)
    # summary(fitM)
    # summary(fitY)
    fitMed <- mediate(fitM, fitY, treat="a", mediator="x")

    # Uncomment to check output
    #print(summary(fitMed))

    #return vector of ACME, ADE, and p_values
    #Note: remember that ACME is estimate for the mediating effect
    return(c(fitMed$d0, fitMed$d0.p, fitMed$z0, fitMed$z0.p))
    
}
print(run_analysis(dat))

[1] 1.4177618 0.0060000 0.5713171 0.0240000


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [12]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)
    
    # Start simulating
    for (i in 1:num_simulations) {
      # WRITE YOUR CODE HERE
      data <- simulate_data(sample_size, age_lo, age_hi, beta_xa,
        beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,] <- run_analysis(data)
     
    }
    
    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[,2] <= alpha)
    ADE_cov =  mean(simouts[,4] <= alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [16]:
# WRITE YOUR CODE HERE
set.seed(2024)

results = repeat_analysis(10, 0.01, sample_size, age_lo, 
                            age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, 
                            beta_c0, sd_c)
print(results)


$ACME_cov
[1] 0.7

$ADE_cov
[1] 0.5



---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [19]:
# WRITE YOUR CODE HERE
# first, create a data.frame to store the results
results_df <- expand.grid(sample_size = c(50, 75, 100, 125, 150))

# iterate over the sample sizes with a for loop
for (i in 1:nrow(results_df)) {
    results <- repeat_analysis(10, 0.01, results_df$sample_size[i], 
        age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, 
        beta_c0, sd_c)
        
    #extract and assign coverage
    results_df$ACME_cov[i] <- results$ACME_cov
    results_df$ADE_cov[i] <- results$ADE_cov
}

Print your results. 

In [20]:
# WRITE YOUR CODE HERE
results_df

sample_size,ACME_cov,ADE_cov
<dbl>,<dbl>,<dbl>
50,0.2,0.7
75,0.8,0.5
100,0.8,0.7
125,1.0,0.8
150,0.9,0.9


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 
 
> *Write your response here.* 
> I would conclude that a sample size of 150 would give the most power for detecting the a mediated effect because largest ACME and ADE coverage of 0.9, 

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *Write your response here.* 

It could be that the parameter used to simulate the moderating effect, beta_cx = 3, is larger than the parameter used to simulate the direct effect beta_ca = 0.8. Therefore, the smaller direct effect was more difficult to detect. 

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*